<a href="https://colab.research.google.com/github/zjzsu2000/CMPE258/blob/master/Ungraded_assignment_5/1)_tensoflow_redo1_quickstart_advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#####Ref:https://www.tensorflow.org/tutorials/quickstart/advanced

# Redo_ TensorFlow_2_quickstart_advanced

##Import TensorFlow

In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

##Load MNIST dataset 

In [2]:
data = tf.keras.datasets.mnist

(X_train, y_train), (X_test, y_test) = data.load_data()


11493376/11490434 [==============================] - 0s 0us/step


In [0]:
#scaling
X_train, X_test = X_train / 255.0, X_test / 255.0


In [0]:
# Add  channels dimension
X_train = X_train[..., tf.newaxis]
X_test = X_test[..., tf.newaxis]

##Batch and shuffle the dataset:

In [0]:
train = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(10000).batch(32)

test = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)

##Build model

In [0]:
class BiuldModel(Model):
  def __init__(self):
    super(BiuldModel, self).__init__()
    self.conv_layer = Conv2D(32,3,activation='relu')
    self.flatten_layer = Flatten()
    self.dense1 = Dense(128,activation='relu')
    self.dense2 = Dense(10)

  def call(self, X):
    X = self.conv_layer(X)
    X = self.flatten_layer(X)
    X = self.dense1(X)
    return self.dense2(X)

## Create instance

In [0]:
model = BiuldModel()

##Choose optimizer and loss function

In [0]:
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = keras.optimizers.Adam()

## Select metrics

In [0]:
train_loss = keras.metrics.Mean(name='train_loss')
train_accuracy =keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = keras.metrics.Mean(name='test_loss')
test_accuracy = keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## Train the model by using `tf.GradientTape`

###Train_step function

In [0]:
def train_step(imags, labels):
  with tf.GradientTape() as tape:  
    preds_train = model(imags, training=True)
    loss_train = loss(labels, preds_train)
  grads = tape.gradient(loss_train, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  train_loss(loss_train)
  train_accuracy(labels, preds_train)

 ### model test function

In [0]:
def test_step(imags, labels): 
  preds_test = model(imags, training=False)
  loss_test = loss(labels, preds_test)

  test_loss(loss_test)
  test_accuracy(labels, preds_test)

In [0]:
EPOCHS = 10

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()
  for imgs, labels in train:
    train_step(imgs, labels)

  for test_imgs, test_labels in test:
    test_step(test_imgs, test_labels)

  str_temp = 'Epoch Num {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(str_temp.format(epoch + 1,train_loss.result(),train_accuracy.result() * 100,
                        test_loss.result(),test_accuracy.result() * 100))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch Num 1, Loss: 0.1378888189792633, Accuracy: 95.89666748046875, Test Loss: 0.06520373374223709, Test Accuracy: 97.87999725341797
Epoch Num 2, Loss: 0.04401770606637001, Accuracy: 98.65333557128906, Test Loss: 0.04830833524465561, Test Accuracy: 98.33999633789062
Epoch Num 3, Loss: 0.0237199068069458, Accuracy: 99.25167083740234, Test Loss: 0.05507471039891243, Test Accuracy: 98.38999938964844
Epoch Num 4, Loss: 0.014099923893809319, Accuracy: 99.50167083740234, Test Loss: 0.0516551248729229, Test Ac

## The image classifier is now trained to ~98.6% accuracy.